# Data Extraction

## 1. US Counties COVID 19 Dataset

The New York Times GITHUB source:
[CSV US counties](https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv) (https://www.kaggle.com/fireballbyedimyrnmom/us-counties-covid-19-dataset#us-counties.csv)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Import data

url_corona_case = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'

corona_case = pd.read_csv(url_corona_case)
corona_case.head(6)

In [100]:
# Convert state to abbrev for corona_case

def state_to_abbrev(state):
    us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
    }
    assert state in us_state_abbrev.keys()
    return us_state_abbrev[state]

In [ ]:
n_row = len(corona_case['state'] )
for i in range(0, n_row):
    corona_case['state'][i] = state_to_abbrev(corona_case['state'][i])
corona_case.head()

c:\users\jbmjy\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [44]:
corona_case = corona_case[['date','state','cases','deaths']]
grouped_corona_case = corona_case.groupby(['state', 'date'])
summarized_corona_case = grouped_corona_case.sum()
summarized_corona_case.head()

cases  deaths
state date                     
AK    2020-03-12      1       0
      2020-03-13      1       0
      2020-03-14      1       0
      2020-03-15      1       0
      2020-03-16      3       0

## 2. United States by Density 2020

Website: https://worldpopulationreview.com/states/state-densities/

In [86]:
import requests
from bs4 import BeautifulSoup as bs

url = "https://worldpopulationreview.com/states/state-densities/"

res = requests.get(url)
soup = bs(res.content, 'html.parser')

In [87]:
tables = soup.find_all('table')
len(tables) # only one table exists

1

In [88]:
# header = [th.getText().strip() for th in tables[0].thead.tr.find_all('th')]
header = ['state', 'density', 'pop_2020', 'area_mi']
tbl_rows = tables[0].tbody.find_all('tr')
df_rows = [[td.getText().strip() for td in tr.find_all('td')] for tr in tbl_rows]
pops = pd.DataFrame(df_rows, columns=header)

In [89]:
n_row = len(pops['state'] )
for i in range(0, n_row):
    pops['state'][i] = state_to_abbrev(pops['state'][i])
pops.head()

,state,density,pop_2020,area_mi
0,DC,"11,815","720,687",61
1,NJ,"1,215","8,936,574","7,354"
2,RI,"1,021","1,056,161","1,034"
3,MA,894,"6,976,597","7,800"
4,CT,736,"3,563,077","4,842"


In [90]:
for i in range(0, n_row):
    pops['density'][i] = pops['density'][i].replace(',','')
    pops['pop_2020'][i] = pops['pop_2020'][i].replace(',','')
    pops['area_mi'][i] = pops['area_mi'][i].replace(',','')
pops.head()

,state,density,pop_2020,area_mi
0,DC,11815,720687,61
1,NJ,1215,8936574,7354
2,RI,1021,1056161,1034
3,MA,894,6976597,7800
4,CT,736,3563077,4842


In [92]:
for col in ['density', 'pop_2020', 'area_mi']:
    pops[col] = pops[col].astype(float)
pops.head()

,state,density,pop_2020,area_mi
0,DC,11815.0,720687.0,61.0
1,NJ,1215.0,8936574.0,7354.0
2,RI,1021.0,1056161.0,1034.0
3,MA,894.0,6976597.0,7800.0
4,CT,736.0,3563077.0,4842.0


In [93]:
# Normalization

min = pops['density'].min()
max = pops['density'].max()

def norm(col):
    x = col.copy()
    n_row = len(x)
    for i in range(0, n_row):
        x[i] = (x[i] - min) / (max - min)
    return x

In [94]:
pops['density_norm'] = norm(pops['density'])
pops.head()


,state,density,pop_2020,area_mi,density_norm
0,DC,11815.0,720687.0,61.0,1.000000
1,NJ,1215.0,8936574.0,7354.0,0.102759
2,RI,1021.0,1056161.0,1034.0,0.086338
3,MA,894.0,6976597.0,7800.0,0.075588
4,CT,736.0,3563077.0,4842.0,0.062214


In [98]:
summarized_corona_case = summarized_corona_case.reset_index()
corona_case_pop = pd.merge(summarized_corona_case, pops,  how='left', left_on=['state'], right_on = ['state'])
corona_case_pop.head()

ValueError: cannot insert level_0, already exists

In [96]:
corona_case_pop.reset_index()

















ValueError: cannot insert level_0, already exists

## b. US Zipcodes to County State to FIPS Crosswalk

Kaggle data source: https://www.kaggle.com/danofer/zipcodes-county-fips-crosswalk

In [7]:
code_mapping = pd.read_csv('ZIP-COUNTY-FIPS_2017-06.csv')
code_mapping.head(6)




,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP
0,36003,Autauga County,AL,1001,H1
1,36006,Autauga County,AL,1001,H1
2,36067,Autauga County,AL,1001,H1
3,36066,Autauga County,AL,1001,H1
4,36703,Autauga County,AL,1001,H1
5,36701,Autauga County,AL,1001,H1
